In [21]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using Random

In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    
    x_prev = x_prior
    
    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [22]:
seed = 123

rng = MersenneTwister(seed)

P = 1.0
n = 500

data = collect(1:n) + rand(rng, Normal(0.0, sqrt(P)), n);

In [23]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, 1.0, P);

    x_buffer  = buffer(Marginal, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, x_buffer)
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return getvalues(x_buffer)
end

inference (generic function with 1 method)

In [24]:
x0_prior = NormalMeanVariance(0.0, 10000.0)

NormalMeanVariance{Float64}(μ=0.0, v=10000.0)

In [25]:
@benchmark res = inference($data, $x0_prior, $P)

BechmarkTools.Trial: 248 samples with 1 evaluations.
 Range (min … max):  15.752 ms … 31.217 ms  ┊ GC (min … max):  0.00% … 35.34%
 Time  (median):     17.906 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   20.155 ms ±  4.293 ms  ┊ GC (mean ± σ):  12.62% ± 15.43%

   ▂▂▂▄▄██▅  ▁                                                 
  ▅█████████▆█▆█▅▅▆▃▁▃▁▁▁▁▁▃▁▁▃▁▁▁▁▁▃▄▃▁▁▆▄▆▃▇▅▄▃▆▇▅▃▅▁▃▃▁▃▃▃ ▃
  15.8 ms         Histogram: frequency by time        29.8 ms <

 Memory estimate: 12.33 MiB, allocs estimate: 207093.

In [26]:
inference(data, x0_prior, P)

500-element Vector{Marginal}:
 Marginal(NormalMeanVariance{Float64}(μ=1.0407099006400038, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=2.040709900640004, v=0.0019999996000000823))
 Marginal(NormalMeanVariance{Float64}(μ=3.0407099006400036, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=4.040709900640002, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=5.040709900640003, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=6.040709900640003, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=7.040709900640005, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=8.040709900640003, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=9.040709900640003, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=10.040709900640003, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=11.040709900640001, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float6